In [2]:
import pandas as pd

## read file
def read_file(year,month,day):
    data = []
    if len(str(month)) == 1:
        month_ = '0'+str(month)
    else:
        month_ = str(month)
    if len(str(day)) == 1:
        day_ = '0'+str(day)
    else:
        day_ = str(day)
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_'+str(year)+'.'+str(month_)+'.'+str(day_)+'.csv'
    data = pd.read_csv(datapath)
    data_CN14 = data[data.Series == data['Series'][0]]
    data = data_CN14
    return data

In [3]:
data = read_file(2014,1,30)

In [226]:
import time 
import csv
with open('order_book.csv', 'wb') as csvfile:
    f = csv.writer(csvfile)
def insert(order_book_data,data_to_insert,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position:]
    return pd.concat((top,data_to_insert,bottom)).reset_index(drop = True)
def draw_out(order_book_data,ob_position):
    top = order_book_data[0:ob_position]
    bottom = order_book_data[ob_position+1:]
    return pd.concat((top,bottom)).reset_index(drop = True)
data['QuantityDifference_'] = data['QuantityDifference']
data_ask = data[(data.BidOrAsk == 'A')].reset_index(drop=True)
data_bid = data[(data.BidOrAsk == 'B')].reset_index(drop=True)
start  =  time.time()
order_book_bid = []
order_book_ask = []
timestamp = data.TimeStamp[0]
temp_ask = 0
temp_bid = 0
bid = []
ask = []
timestamp_ = []
x1 = data[(data.BidOrAsk == 'A')].TimeStamp.unique()
x2 = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
y = data[data['TimeStamp'].str.contains(timestamp)]
bid.append(y[(y.BidOrAsk == 'B')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # bid
ask.append(y[(y.BidOrAsk == 'A')][["Price","OrderNumber","QuantityDifference","QuantityDifference_"]]) # ask
a = bid[0].sort(['Price'],ascending = [False])
b = ask[0].sort(['Price'],ascending = [True])    
order_book_bid.append(a[a.QuantityDifference != 0].reset_index(drop = True))
order_book_ask.append(b[b.QuantityDifference != 0].reset_index(drop = True))
order_book_bid.append(0)
order_book_ask.append(0)
order_book_bid_sum = order_book_bid[0][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False,sort = False).sum()
order_book_ask_sum = order_book_ask[0][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()
order_book_bid_ask = pd.concat([order_book_bid_sum[['Price','QuantityDifference']],order_book_ask_sum[['Price','QuantityDifference']]],axis = 1)

with open('order_book.csv','a') as f:
    order_book = csv.writer(f)
    order_book.writerow(["Bid","Bid_Quantity","Ask","Ask_Quantity"])
    order_book.writerow(["TimeStamp",timestamp])
    order_book = csv.writer(f,delimiter=',')
    for i in range(0,max(len(order_book_ask[0]),len(order_book_bid[0]))):
        order_book.writerow(order_book_bid_ask[i:i+1].values.tolist()[0])
        

/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:30: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/bigdatas16/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:31: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [227]:
def order_book_to_csv(order_book_bid,order_book_ask,data,i):
    order_book_bid_sum = order_book_bid[['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False,sort = False).sum()
    order_book_ask_sum = order_book_ask[['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()
    order_book_bid_ask = pd.concat([order_book_bid_sum[['Price','QuantityDifference']],order_book_ask_sum[['Price','QuantityDifference']]],axis = 1)
    print order_book_bid_ask
    with open('order_book.csv','a') as f:
        order_book = csv.writer(f)
        order_book.writerow(["TimeStamp",data.TimeStamp[i:i+1].iloc[0]])
        order_book = csv.writer(f,delimiter=',')
        for i in range(0,len(order_book_bid_ask)):
            order_book.writerow(order_book_bid_ask[i:i+1].values.tolist()[0])


In [228]:
for i in range(len(y),30,1):
    
    print i#,temp_bid,temp_ask
    time_second = int(data[i:i+1].TimeStamp.iloc[0][18]) + int(data[i:i+1].TimeStamp.iloc[0][17])*10 +\
                  int(data[i:i+1].TimeStamp.iloc[0][15])*60 + int(data[i:i+1].TimeStamp.iloc[0][14])*600 +\
                  int(data[i:i+1].TimeStamp.iloc[0][12])*3600 + int(data[i:i+1].TimeStamp.iloc[0][11])*36000
    if time_second == 32400:
        order_book_bid[temp_bid] = order_book_bid[temp_bid].sort(['Price'],ascending = [False]).reset_index(drop = True)
        order_book_ask[temp_ask] = order_book_ask[temp_ask].sort(['Price'],ascending = [True]).reset_index(drop = True)
        pass
    
    if data.BidOrAsk[i] == 'A':
        if int(data[['QuantityDifference']][i:i+1].values) > 0 :
            if order_book_bid[temp_bid].Price[0] >= data[i:i+1].Price.iloc[0] and time_second < 32400:
                for k in range(0,len(order_book_bid[temp_bid])):
                    diff = order_book_bid[temp_bid].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                    if order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff >= 0:
                        order_book_bid[temp_bid].QuantityDifference_[k] = diff
                        data[i:i+1].QuantityDifference_.iloc[0] = 0
                        break
                    elif order_book_bid[temp_bid].Price[k] >= data[i:i+1].Price.iloc[0] and diff < 0:
                        order_book_bid[temp_bid].QuantityDifference_[k] = 0
                        data[i:i+1].QuantityDifference_.iloc[0] = -diff
                        pass
                    else:
                        break
            if data.TimeStamp[i] == x1[temp_ask]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                length_order_book_zero = len(order_book_ask[temp_ask][(order_book_ask[temp_ask].Price <= data['Price'][i:i+1].iloc[0]) & (order_book_ask[temp_ask].QuantityDifference_ == 0)])
                if time_second > 32400:
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:  
                            print 'Some error1(Ask & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< (len(order_book_ask[temp_ask])-1):
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:#or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1(Ask & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'   
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:
                        if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 
                            print 'Some error1(Ask & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True' 
                        pass
                else:
                    print 'before open market'
            elif data.TimeStamp[i] != x1[temp_ask]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                if temp_ask == 0:
                    order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = order_book_ask[temp_ask - 1]
                    best_price = data[i:(i+1)]['BestPrice']
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                  
                    if time_second > 32400:
                        if position_ == 0 and len(order_book_ask[0]) > 1:
                            if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 

                                print 'Some error2(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass
                        elif 0 < position_< len(order_book_ask[temp_ask])-1:
                            if order_book_ask[temp_ask][position_+ 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_ask[temp_ask][position_ -  1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 

                                print 'Some error2(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_ask[temp_ask])-1:   
                            if order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: #or order_book_ask[temp_ask]['Price'][0:1].iloc[0] != data['BestPrice'][i]: 

                                print 'Some error2(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                print 'position and Best Price is True'
                                pass
                        elif position_ == 0 and len(order_book_ask[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                    else:
                        print 'before open market'
                else:
                    order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    order_book_ask.append(order_book_ask[temp_ask])
                    temp_ask = temp_ask + 1
                    order_book_ask[temp_ask] = insert(order_book_ask[temp_ask],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)#int(data[['OrderBookPosition']][i:i+1].iloc[0])-1)
                    if time_second > 32400:
                        if position_ == 0:
                            if order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                                print 'Some error3(Ask & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass
                        elif 0 < position_< len(order_book_ask[temp_ask])-1:
                            if order_book_ask[temp_ask][position_+1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                                print 'Some error3(Ask & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_ask[temp_ask])-1:    
                            if order_book_ask[temp_ask][position_-1:position_-1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 

                                print 'Some error3(Ask & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                print 'position and Best Price is True'                        
                                pass
                        elif position_ == 0 and len(order_book_ask[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                    else:
                        print 'before open market'
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    
            if data.TimeStamp[i] == x1[temp_ask]: 
                print 'timestamp = %s'%(data.TimeStamp[i])
                order_number_ =  data['OrderNumber'][i:i + 1].iloc[0]
                position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                if time_second > 32400:
                    if position_ == 0 and len(order_book_ask[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                            print 'Some error4(Ask & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass       
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                            print 'Some error4(Ask & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:       
                        if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 

                            print 'Some error4(Ask & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])   
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass                        
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        pass
                else:
                    print 'before open market'                   
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
            elif data.TimeStamp[i] != x1[temp_ask]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
                order_book_ask.append(order_book_ask[temp_ask])
                order_number_ =  data['OrderNumber'][i : i + 1].iloc[0]
                position_ = order_book_ask[temp_ask][order_book_ask[temp_ask].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_ask = temp_ask + 1
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                            print 'Some error5(Ask & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])    
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_ask[temp_ask])-1:    
                        if order_book_ask[temp_ask][position_ + 1:position_+1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]: 

                            print 'Some error5(Ask & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_ask[temp_ask])-1:

                        if position_ > 0 and order_book_ask[temp_ask][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 

                            print 'Some error5(Ask & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass                       
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_ask[0]) == 1:
                        print 'position and Best Price is True'
                        pass      
                else:
                    print 'before open market'
                    
                if order_book_ask[temp_ask][(order_book_ask[temp_ask].OrderNumber == order_number_)&(order_book_ask[temp_ask].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_ask[temp_ask] = order_book_ask[temp_ask].drop(order_book_ask[temp_ask].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] = order_book_ask[temp_ask]['QuantityDifference'][order_book_ask[temp_ask].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
                    
    elif data.BidOrAsk[i] == 'B':
        if int(data[['QuantityDifference']][i:i+1].values) > 0: 
            if order_book_ask[temp_ask].Price[0] <= data[i:i+1].Price.iloc[0] and time_second < 32400:
                for k in range(0,len(order_book_ask[temp_ask])):
                    diff = order_book_ask[temp_ask].QuantityDifference_[k] - data[i:i+1].QuantityDifference_.iloc[0]
                    if order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff >= 0:
                        order_book_ask[temp_ask].QuantityDifference_[k] = diff
                        data[i:i+1].QuantityDifference_.iloc[0] = 0
                        break
                    elif order_book_ask[temp_ask].Price[k] <= data[i:i+1].Price.iloc[0] and diff < 0:
                        order_book_ask[temp_ask].QuantityDifference_[k] = 0
                        data[i:i+1].QuantityDifference_.iloc[0] = - diff
                        pass
                    else:
                        break
            if data.TimeStamp[i] == x2[temp_bid]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)
                length_order_book_zero = len(order_book_bid[temp_bid][(order_book_bid[temp_bid].Price >= data['Price'][i:i+1].iloc[0]) & (order_book_bid[temp_bid].QuantityDifference_ == 0)])
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:                
                            print 'Some error1(Bid & Q>0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass     
                    elif 0 < position_< len(order_book_bid[temp_bid])-1: 
                        if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]: 
                            print 'Some error1(Bid & Q>0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break    
                        else:
                            print 'position and Best Price is True'   
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1 and len(order_book_bid[0]) > 1:      
                        if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            print 'Some error1(Bid & Q>0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass 
                else:
                    print 'before open market'
                    pass
            elif data.TimeStamp[i] != x2[temp_bid]:
                print temp_bid
                print 'timestamp = %s'%(data.TimeStamp[i])
                if temp_bid == 0:
                    order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
                    best_price = data[i:(i+1)]['BestPrice']             
                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    temp_bid = temp_bid + 1
                    order_book_bid[temp_bid] = order_book_bid[temp_bid - 1]
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)                 
                    if time_second > 32400:
                        if position_ == 0  and len(order_book_bid[0]) > 1:
                            if order_book_bid[temp_bid][position_ + 1:position_ + 1 + 1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error2(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0]) + data['OrderNumber'][i:i+1].iloc[0]
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass       
                        elif 0 < position_< len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                                print 'Some error2(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_bid[temp_bid])-1:    
                            if order_book_bid[temp_bid][position_ -  1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error2(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])                                        
                                pass
                            else:
                                print 'position and Best Price is True'
                                pass
                        elif position_ == 0 and len(order_book_bid[0]) == 1:
                                print 'position and Best Price is True'
                                pass
                    else:
                        print 'before open market'
                        pass
                else:
                    order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)

                    position_ = int(data[['OrderBookPosition']][i:i+1].iloc[0]) - 1 
                    order_book_bid.append(order_book_bid[temp_bid])
                    temp_bid = temp_bid + 1 
                    order_book_bid[temp_bid] = insert(order_book_bid[temp_bid],data[['Price','OrderNumber','QuantityDifference','QuantityDifference_']][i:i+1],position_)   
                    if time_second > 32400:
                        if position_ == 0 and len(order_book_bid[0]) > 1:
                            if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error3(Bid & Q>0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])     
                                break
                            else:
                                print 'position and Best Price is True'                     
                                pass
                        elif 0 < position_< len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_-1:position_-1+1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_+1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                                print 'Some error3(Bid & Q>0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                                break
                            else:
                                print 'position and Best Price is True'  
                                pass
                        elif position_ == len(order_book_bid[temp_bid])-1:
                            if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid]['Price'][0:1].iloc[0] != data['BestPrice'][i]:
                                print 'Some error3(Bid & Q>0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                                break
                            else:
                                print 'position and Best Price is True'                        
                                pass
                        elif position_ == 0 and len(order_book_bid[0]) == 1:
                            print 'position and Best Price is True'
                            pass
                    else:
                        print 'before open market'
                        pass
        elif int(data[['QuantityDifference']][i:i+1].values) < 0:    
            if data.TimeStamp[i] == x2[temp_bid]: 
                print 'timestamp = %s'%(data.TimeStamp[i])
                order_number_ =  data['OrderNumber'][i : i + 1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error4(Bid & Q<0 & timestamp not change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass               
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error4(Bid & Q<0 & timestamp not change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1:
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            print 'Some error4(Bid & Q<0 & timestamp not change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])      
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass
                        else:
                            print 'position and Best Price is True'                        
                            pass
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass
                else:
                    print 'before open market'
                    pass
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]                
            elif data.TimeStamp[i] != x2[temp_bid]:
                print 'timestamp = %s'%(data.TimeStamp[i])
                order_book_to_csv(order_book_bid[temp_bid],order_book_ask[temp_ask],data,i)
        
                order_book_bid.append(order_book_bid[temp_bid])
                order_number_ =  data['OrderNumber'][i:i+1].iloc[0]
                position_ = order_book_bid[temp_bid][order_book_bid[temp_bid].OrderNumber == order_number_].index[0]
                price_ = data[i:i+1]['Price'].iloc[0]
                temp_bid = temp_bid + 1
                if time_second > 32400:
                    if position_ == 0 and len(order_book_bid[0]) > 1:
                        if order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error5(Bid & Q<0 & timestamp change & 1),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'                     
                            pass
                    elif 0 < position_< len(order_book_bid[temp_bid])-1:    
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0] or order_book_bid[temp_bid][position_ + 1:position_+1+1]["Price"].iloc[0] > data['Price'][i:i+1].iloc[0]:
                            print 'Some error5(Bid & Q<0 & timestamp change & 2),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        else:
                            print 'position and Best Price is True'  
                            pass
                    elif position_ == len(order_book_bid[temp_bid])-1: 
                        if order_book_bid[temp_bid][position_ - 1:position_ - 1 + 1]["Price"].iloc[0] < data['Price'][i:i+1].iloc[0]:
                            print 'Some error5(Bid & Q<0 & timestamp change & 3),position = %d,index = %d,price = %d,OrderNumber = %s'%(position_,i,data['Price'][i:i+1].iloc[0],data['OrderNumber'][i:i+1].iloc[0])
                            break
                        elif position_ == 0:
                            print 'position and Best Price is True'
                            pass
                        else:
                            print 'position and Best Price is True'                        
                            pass    
                    elif position_ == 0 and len(order_book_bid[0]) == 1:
                        print 'position and Best Price is True'
                        pass  
                else:
                    print 'before open market'
                    pass
                if order_book_bid[temp_bid][(order_book_bid[temp_bid].OrderNumber == order_number_)&(order_book_bid[temp_bid].Price == price_)]['QuantityDifference'].iloc[0] == abs(data[i:i+1]['QuantityDifference'].iloc[0]):
                    order_book_bid[temp_bid] = order_book_bid[temp_bid].drop(order_book_bid[temp_bid].index[[position_]]).reset_index(drop = True)
                else:
                    order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] = order_book_bid[temp_bid]['QuantityDifference'][order_book_bid[temp_bid].OrderNumber == order_number_] + data[i:i+1]['QuantityDifference'].iloc[0]
end = time.time()            
print "Total time = %f"%(end - start)

13
0
timestamp = 2014-01-30D08:45:00.361550600
      Price  QuantityDifference   Price  QuantityDifference
0  665000.0               150.0  682000                   1
1  663500.0                 1.0  683500                   1
2  663000.0                10.0  689000                 100
3  660000.0               100.0  691000                  10
4       NaN                 NaN  699000                  10
5       NaN                 NaN  700000                   1
6       NaN                 NaN  717500                  10
before open market
14
1
timestamp = 2014-01-30D08:45:01.844526600
      Price  QuantityDifference   Price  QuantityDifference
0  668000.0                 1.0  682000                   1
1  665000.0               150.0  683500                   1
2  663500.0                 1.0  689000                 100
3  663000.0                10.0  691000                  10
4  660000.0               100.0  699000                  10
5       NaN                 NaN  700000        

In [229]:
pd.read_table('order_book.csv',sep=',')

,Bid,Bid_Quantity,Ask,Ask_Quantity
0,TimeStamp,2014-01-30D04:27:08.581903900,NaN,NaN
1,665000.0,150.0,682000.0,1.0
2,663500.0,1.0,683500.0,1.0
3,663000.0,10.0,689000.0,100.0
4,660000.0,100.0,691000.0,10.0
5,NaN,NaN,699000.0,10.0
6,NaN,NaN,700000.0,1.0
7,NaN,NaN,717500.0,10.0
8,TimeStamp,2014-01-30D08:45:00.361550600,NaN,NaN
9,665000.0,150.0,682000.0,1.0


In [172]:
data.head(30)

,Series,SequenceNumber,TimeStamp,OrderNumber,OrderBookPosition,Price,QuantityDifference,Trade,BidOrAsk,BestPrice,BestQuantity,QuantityDifference_
0,CNG14,1,2014-01-30D04:27:08.581903900,BFC92EA46AE44FA4,1,700000,1,,A,682000,1,1
1,CNG14,2,2014-01-30D04:27:08.581903900,0F50C53613C0F881,1,682000,1,,A,682000,1,1
2,CNG14,3,2014-01-30D04:27:08.581903900,6EA6E34859783E78,2,691000,10,,A,682000,1,10
3,CNG14,4,2014-01-30D04:27:08.581903900,48714F4C6C254E45,1,665000,50,,B,665000,150,50
4,CNG14,5,2014-01-30D04:27:08.581903900,C0F94F4C3F5B217B,1,665000,50,,B,665000,150,50
5,CNG14,6,2014-01-30D04:27:08.581903900,6CB163000307E808,3,660000,100,,B,665000,150,100
6,CNG14,7,2014-01-30D04:27:08.581903900,662E6B4847A22CA2,3,699000,10,,A,682000,1,10
7,CNG14,8,2014-01-30D04:27:08.581903900,2BD5C16638561AB6,5,717500,10,,A,682000,1,10
8,CNG14,9,2014-01-30D04:27:08.581903900,D8F8E619640F43AF,2,683500,1,,A,682000,1,1
9,CNG14,10,2014-01-30D04:27:08.581903900,C4F76F586C344E54,3,689000,100,,A,682000,1,100


In [179]:
temp_ask,temp_bid

(2, 14)

In [180]:
order_book_ask[0],order_book_bid[0]

(    Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  682000  0F50C53613C0F881                   1                    1
 1  683500  D8F8E619640F43AF                   1                    1
 2  689000  C4F76F586C344E54                 100                  100
 3  691000  6EA6E34859783E78                  10                   10
 4  699000  662E6B4847A22CA2                  10                   10
 5  700000  BFC92EA46AE44FA4                   1                    1
 6  717500  2BD5C16638561AB6                  10                   10,
     Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  665000  48714F4C6C254E45                  50                   50
 1  665000  C0F94F4C3F5B217B                  50                   50
 2  665000  4071CF4C6EE55105                  50                   50
 3  663500  FE234A199A1979B9                   1                    1
 4  663000  27586D726EF753F7                  10                   10
 5  660000  6CB1630

In [181]:
order_book_ask[1],order_book_bid[1]

(    Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  682000  0F50C53613C0F881                   1                    1
 1  683500  D8F8E619640F43AF                   1                    1
 2  689000  C4F76F586C344E54                 100                  100
 3  691000  6EA6E34859783E78                  10                   10
 4  699000  662E6B4847A22CA2                  10                   10
 5  700000  BFC92EA46AE44FA4                   1                    1
 6  717500  2BD5C16638561AB6                  10                   10
 7  725000  7A666A457D4F5C2F                   4                    4,
     Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  668000  2DD0417EFB04DFC4                   1                    1
 1  665000  48714F4C6C254E45                  50                   50
 2  665000  C0F94F4C3F5B217B                  50                   50
 3  665000  4071CF4C6EE55105                  50                   50
 4  663500  FE234A1

In [182]:
order_book_ask[2],order_book_bid[2]

(    Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  682000  0F50C53613C0F881                   1                    1
 1  682000  83A8692BEE72D0B2                   1                    1
 2  683500  D8F8E619640F43AF                   1                    1
 3  689000  C4F76F586C344E54                 100                  100
 4  691000  6EA6E34859783E78                  10                   10
 5  699000  662E6B4847A22CA2                  10                   10
 6  700000  BFC92EA46AE44FA4                   1                    1
 7  717500  2BD5C16638561AB6                  10                   10
 8  725000  7A666A457D4F5C2F                   4                    4,
     Price       OrderNumber  QuantityDifference  QuantityDifference_
 0  668000  2DD0417EFB04DFC4                   1                    1
 1  665500  8142C93656273847                   1                    1
 2  665000  48714F4C6C254E45                  50                   50
 3  665000  C0F94F4

In [32]:
order_book_ask[temp_ask],order_book_bid[temp_bid]

(      Price       OrderNumber  QuantityDifference  QuantityDifference_
 0    668000  5872EF19009DE4DE                   5                   10
 1    668000  0B88CD76947A795A                  10                   10
 2    668000  23346D6BF357D597                   1                    1
 3    668000  2B114566DBD1C091                   1                    1
 4    668000  DEA1C3197F185E58                  10                   10
 5    668000  F63A621D1E32FDF3                  78                   78
 6    668000  F43AEA1DEA94C974                  20                   20
 7    668000  52268A8986FD677D                   8                    8
 8    668000  8DF0E563D3DFB61F                   8                    8
 9    668000  5AA60A81376A17EA                   8                    8
 10   668000  D03DE65D415B229B                   2                    2
 11   668000  F4320EC980AB612B                   8                    8
 12   668500  A382C93E4E1B331B                  45              

In [52]:
order_book_ask[0][['Price','QuantityDifference']][1:2].values.tolist()[0]

[683500, 1]

In [25]:
import csv
import sys

In [74]:
import csv
with open('order_book.txt', 'wb') as csvfile:
    f = csv.writer(csvfile)#, delimiter=' ',quotechar='|', quoting=csv.QUOTE_MINIMAL)

In [53]:
with open('eggs.txt','a') as f:
    spamwriter = csv.writer(f,delimiter=',',)
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][0:1].values.tolist()[0])
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][1:2].values.tolist()[0])
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][2:3].values.tolist()[0])
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][3:4].values.tolist()[0])
    spamwriter.writerow(order_book_ask[0][['Price','QuantityDifference']][4:5].values.tolist()[0])

In [96]:
with open('order_book.txt','a') as f:
    order_book = csv.writer(f)
    order_book.writerow([data.TimeStamp[i]])
    order_book = csv.writer(f,delimiter=',')
    for i in range(1,max(len(order_book_ask[0]),len(order_book_bid[0]))):
        order_book.writerow(a[i:i+1].values.tolist()[0])
        

In [81]:
data.TimeStamp[0]

'2014-01-30D04:27:08.581903900'

In [135]:
#a = pd.concat([order_book_bid[1][['Price','QuantityDifference']],order_book_ask[1][['Price','QuantityDifference']]],axis = 1)

In [134]:
#a

In [133]:
#pd.read_table('eggs.txt')#,sep = ',')

In [19]:
with open('order_book.csv', 'rb') as f:
    reader = csv.reader(f)
    for row in reader:
        print row

In [5]:
import time
start = time.time()
order_book_bid_merge = []
order_book_ask_merge = []
for i in range(0,len(order_book_bid)):
    a = order_book_bid[i][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()#.sort(['Price'],ascending = [False])
    order_book_bid_merge.append(pd.DataFrame(a.values))
for i in range(0,len(order_book_ask)):
    a = order_book_ask[i][['Price','QuantityDifference']].groupby(by = ['Price'],as_index = False).sum()#.sort(['Price'],ascending = [True])
    order_book_ask_merge.append(pd.DataFrame(a.values))
end = time.time()            
print ("Total time = %f"%(end - start)) 


Total time = 0.432648


In [17]:
order_book_bid_merge[7]

,0,1
0,660000,100
1,663000,10
2,663500,1
3,664000,1
4,665000,151
5,665500,1
6,666000,1
7,668000,1


In [10]:
order_book_ask_merge[0]

,0,1
0,682000,1
1,683500,1
2,689000,100
3,691000,10
4,699000,10
5,700000,1
6,717500,10


In [20]:
import time
start = time.time()
time_ = data.TimeStamp.unique()
time_bid = data[(data.BidOrAsk == 'B')].TimeStamp.unique()[1:]
time_ask = data[(data.BidOrAsk == 'A')].TimeStamp.unique()[1:]
bid_index = 0
ask_index = 0
a = 0
b = 0
first = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
first = pd.DataFrame(first.values)
time = pd.DataFrame([['TimeStamp',time_[0][11:29],'','']])
order_book_concat = time.append(first).reset_index(drop=True)
order_book = order_book_concat
#print order_book
time_series = 1
for time in time_[1:]:
    #print 'time' + time
    #print 'time_series = %d'%(time_series)
    time_to_concat = pd.DataFrame([['TimeStamp',time_[time_series][11:29],'','']])
    #print time_to_concat
    #print time_bid[a]
    #print time_ask[b]
    #print order_book_bid_merge[a]
    #print order_book_ask_merge[b]
    if time == time_bid[a] and time == time_ask[b]:
        #print '1 = ' + time
        bid_index = bid_index + 1
        ask_index = ask_index + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
        b = b + 1
    elif time == time_bid[a] and time != time_ask[b]: 
        #print '2 = ' +time
        bid_index = bid_index + 1        
        
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
    elif time != time_bid[a] and time == time_ask[b]: 
        #print '3 = ' +time
        ask_index = ask_index + 1    
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        b = b + 1
    else:
        continue
    order_book = pd.concat([order_book,order_book_concat])    
    time_series = time_series + 1    
    
    #print order_book
end = time.time()            
print "Total time = %f"%(end - start) 


IndexError: list index out of range

In [31]:
order_book = pd.DataFrame(order_book.values)
order_book.to_csv('CNF14_01_03_order_book_final.csv',sep = '\t',encoding = 'utf-8')

In [30]:
import time
start = time.time()


time_ = data.TimeStamp.unique()
time_bid = data[(data.BidOrAsk == 'B')].TimeStamp.unique()
time_ask = data[(data.BidOrAsk == 'A')].TimeStamp.unique()

bid_index = 0
ask_index = 0
a = 0
b = 0
first = pd.concat([pd.DataFrame(order_book_bid_merge[bid_index].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[ask_index].iloc[0:3].values)],axis = 1)
first = pd.DataFrame(first.values)
time = pd.DataFrame([['TimeStamp',time_[0][11:29],'','']])
order_book_concat = time.append(first).reset_index(drop=True)
order_book = order_book_concat
#print order_book
a = 1
b = 1
time_series = 1
for time in time_[1:]:
    #print 'time' + time
    #print 'time_series = %d'%(time_series)
    time_to_concat = pd.DataFrame([['TimeStamp',time_[time_series][11:29],'','']])
    #print time_to_concat
    #print time_bid[a-1]
    #print time_ask[b-1]    
    #print time_bid[a]
    #print time_ask[b]
    #print order_book_bid_merge[a]
    #print order_book_ask_merge[b]
    if time == time_bid[a] and time == time_ask[b]:
        #print '1 = ' + time
        #a = a + 1
        #b = b + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[a].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[b].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
        b = b + 1
    elif time == time_bid[a] and time != time_ask[b]: 
        #print '2 = ' +time
        ##a = a + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[a].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[b-1].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        a = a + 1
        
    elif time != time_bid[a] and time == time_ask[b]: 
        #print '3 = ' +time
        ##b = b + 1
        order_book_no_time = pd.concat([pd.DataFrame(order_book_bid_merge[a-1].iloc[::-1][0:3].values),pd.DataFrame(order_book_ask_merge[b].iloc[0:3].values)],axis = 1)
        order_book_no_time = pd.DataFrame(order_book_no_time.values)
        order_book_concat = time_to_concat.append(order_book_no_time).reset_index(drop=True)        
        b = b + 1
    else:
        continue
    order_book = pd.concat([order_book,order_book_concat])    
    time_series = time_series + 1    
    #print order_book
end = time.time()            
print "Total time = %f"%(end - start) 


IndexError: index 45578 is out of bounds for axis 0 with size 45578

In [7]:
order_book = pd.DataFrame(order_book.values)
order_book.to_csv('CNF14_01_13_order_book_final.csv',sep = '\t',encoding = 'utf-8')